### EXAMPLE 1

In [1]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import(
    RunnablePassthrough,
    RunnableParallel,
    RunnableLambda
)
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

p:\Langchain_learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
llm = ChatGroq(
    model = "openai/gpt-oss-120b",
    api_key=os.getenv("GROK_API_KEY")
)

In [7]:
prompt = ChatPromptTemplate.from_template("explain this {topic} in one sentence")
parser = StrOutputParser()

chain = prompt | llm | parser

In [9]:
res = chain.invoke({"topic":"LCEL"})
print(res)

LCEL (Logic Constraint Expression Language) is a concise, human‑readable DSL that lets you specify logical conditions and constraints (e.g., comparisons, boolean operators, and functions) in a single expression for use in rule‑based or validation systems.


### EXAMPLE 2: RUNNABLEPASSTHROUGH


In [14]:
transulate_prompt = ChatPromptTemplate.from_template("I need only meaning that's it Transulate into to TAMIL : {text}")

In [19]:
t_chain_without = transulate_prompt | llm | parser

In [20]:
res = t_chain_without.invoke({"text":"Hello"})
res

'வணக்கம் (Vanakkam)'

In [17]:
t_chain_with = (
    RunnablePassthrough.assign(
        transulation = transulate_prompt | llm | parser
    )
)

In [21]:
res1 = t_chain_with.invoke({"text":"Hello"})
res1

{'text': 'Hello', 'transulation': 'வணக்கம்'}

 ### EXAMPLE 3: RUNNABLEPARALLEL

In [23]:
summary_chain = (
    ChatPromptTemplate.from_template("Summarize in 5 words: {text}")
    | llm
    | parser
)

sentiment_chain = (
    ChatPromptTemplate.from_template("Sentiment (positive/negative/neutral): {text}")
    | llm 
    | parser
)

keyword_chain = (
    ChatPromptTemplate.from_template("Extract 3 keywords: {text}")
    | llm 
    | parser
)

In [24]:
import time

In [25]:
text = "I absolutely love LangChain! It's amazing and so easy to use."

In [27]:
start = time.time()
summary = summary_chain.invoke({"text": text})
sentiment = sentiment_chain.invoke({"text": text})
keywords = keyword_chain.invoke({"text": text})
print(f"Sequential time: {time.time() - start:.2f}s")

Sequential time: 9.15s


In [32]:
parallel_chain = RunnableParallel(
    summary = summary_chain,
    sentiment = sentiment_chain,
    keyword = keyword_chain
)

start = time.time()
res = parallel_chain.invoke({"text":text})
print(f"Parallel time: {time.time() - start:.2f}s")
res

Parallel time: 7.03s


{'summary': 'Love LangChain: amazing, incredibly easy.',
 'sentiment': 'Positive',
 'keyword': '**Keywords:**  \n1. LangChain  \n2. amazing  \n3. easy to use'}